# Part 0:
## import everything
Run the cell below

In [ ]:
import os
import glob
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
import sys
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
import PIL
from scipy import stats
import matplotlib.animation as animation
import matplotlib.backends.backend_pdf
import mpl_toolkits.axes_grid1.inset_locator as inset
from matplotlib.ticker import FormatStrFormatter, MaxNLocator
from matplotlib.patches import ConnectionPatch
from set_rc_params import set_rc_params
import ROOT


if "__file__" not in dir():
    %matplotlib inline
    %config InlineBackend.close_figures = False

    root=ROOT.root
    
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
    CWD=os.getcwd()
    os.chdir(CommonNoteBookesPath)
    %run UtilityTools.ipynb
    %run Animal_Tags.ipynb
    %run loadRat_documentation.ipynb
    %run plotRat_documentation_1_GeneralBehavior.ipynb
    %run plotRat_documentation_3_KinematicsInvestigation.ipynb
    %run RunBatchRat_3_CompareGroups.ipynb
    %run BatchRatBehavior.ipynb
    currentNbPath=os.path.join(os.path.split(ThisNoteBookPath)[0],'LesionPaper','BreakEffect.ipynb')
    %run $currentNbPath

    os.chdir(CWD)

    logging.getLogger().setLevel(logging.ERROR)
    
    param={
        "goalTime":7,#needed for pavel data only
        "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
        "maxTrialDuration":15,
        "interTrialDuration":10,#None pavel
        "endTrial_frontPos":30,
        "endTrial_backPos":55, 
        "endTrial_minTimeSec":4,
        "cameraSamplingRate":25, #needed for new setup    

        "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
        "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
        "nbJumpMax":100,#200 pavel
        "binSize":0.25,
        #parameters used to preprocess (will override the default parameters)
    }
    Y1,Y2=param['treadmillRange']

    print('os:',OS(),'\nroot:',root,'\nImport successful!')

---
---


# part 1:

# DEFINITIONS

### If you don't know what to do, move to part 2

---

plotting group event effect

In [ ]:
def plot_group_event_effect(root, ax, Profiles, badAnimals=None, TaskParamToPlot='% good trials',
                            nPre=5, nPost=5, dot_plot=True, **kwarg):

    if badAnimals is None:
        badAnimals=[]
    
    
    #getting the data
    _,SessionDict=event_detect(root, Profiles[0], Profiles[1], badAnimals=badAnimals)

    Results,nSessionPre,nSessionPost=event_statistic2(root,
                                                      SessionDict,
                                                      parameter=param,
                                                      redo=False,
                                                      TaskParamToPlot=[TaskParamToPlot])

    assert nPre<=nSessionPre and nPost<=nSessionPost,"fewer sessions available than requested:"
    
    data=np.array(list(Results[TaskParamToPlot].values())).T
    
    x_axis=np.arange(-nSessionPre,nSessionPost)
    x_axis[x_axis>=0]+=1
    x_axis=x_axis[nSessionPre-nPre:nSessionPre+nPost]
    
    plt.plot(x_axis,np.nanmean(data,axis=1)[nSessionPre-nPre:nSessionPre+nPost],'k-o',lw=2, zorder=5)
    
    #plotting individual animals
    if dot_plot:
        ax.plot(x_axis,data[nSessionPre-nPre:nSessionPre+nPost,:],'o--',
                ms=1,lw=.4,alpha=.8, zorder=1, **kwarg)

    
    ax.set_xticks(x_axis)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_bounds(-nPre,nPost)
    ax.set_xlim([-nPre-1,nPost+1])
    
#     ax.set_ylabel(TaskParamToPlot)
#     ax.set_xlabel('Sessions relative to break / lesion')

    return data

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':'10',
              'Speed':'10',
              'Tag':['Control','Control-BackTo10','Control-Late-BackTo10']
              }
    profile2={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':'10',
              'Speed':'10',
              'Tag':['Control-AfterBreak']
              }

    #number of sessions to plot
    Profiles=(profile1,profile2,)
    TaskParamToPlot='% good trials'
    plt.close('all')
    fig=plt.figure(figsize=(10,4))
    
    ax=fig.add_subplot(111)
    a=plot_group_event_effect(root, ax, Profiles,nPre=2, nPost=2,TaskParamToPlot=TaskParamToPlot)
    ax.set_ylim([0,100])

------



------

# part 2:

# GENERATING THE FIGURE

Definition of Parameters

In [ ]:
if "__file__" not in dir():
    # GENERAL PARAMS
    
    
    #===============================================
    
    # GRID 1 PARAMS
    
    TaskParamToPlot1="% good trials" 
    
    profileCtrlPre1={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':'10',
              'Speed':'10',
              'Tag':['Control','Control-BackTo10','Control-Late-BackTo10']
              }
    profileCtrlPost1={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':'10',
              'Speed':'10',
              'Tag':['Control-AfterBreak']
              }  
    ProfilesCtrl1=(profileCtrlPre1,profileCtrlPost1,)
    nPre1=5
    nPost1=5
    
    #================================================
    
    # GRID 2 PARAMS
    
    TaskParamToPlot2="Motor Sequence Score"
    
    
    #================================================
    
    # GRID 3 PARAMS
    
    TaskParamToPlot3="Forward Running Speed"

    

Plotting the figure

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc_params()
    figsize=(4,5)
    fig=plt.figure(figsize=figsize,dpi=600)
    gs= fig.add_gridspec(nrows=3, ncols=1, left=0.0, bottom=0.0, right=1, top=1)
    
    ##########################################
    # 1: % correct trials
    ax1= fig.add_subplot(gs[0])
    plot_group_event_effect(root, ax1, ProfilesCtrl1, nPre=nPre1, nPost=nPost1,TaskParamToPlot=TaskParamToPlot1)    
    ax1.set_ylim([-1,101])
    ax1.spines['left'].set_bounds(0,100)
    ax1.set_ylabel('% correct trials')

    #adding the vertical line + 'Break' text
    ax1.figure.add_artist(ConnectionPatch(xyA=(.5,gs.top), xyB=(.5,gs.bottom), 
                                         coordsA='figure fraction', coordsB='figure fraction',
                                         ls=':',color='r',lw=2,zorder=10))

    ax1.figure.add_artist(mpl.text.Text(x=.5, y=gs.top, text='Break', c='k',ha='center',va='bottom',fontsize='x-small'))

    
    
    ##########################################
    # 2: % sequential score
    ax2= fig.add_subplot(gs[1])
    
    plot_group_event_effect(root, ax2, ProfilesCtrl1, nPre=nPre1, nPost=nPost1,TaskParamToPlot=TaskParamToPlot2)
    
    ax2.set_ylim([-.01,1.01])
    ax2.spines['left'].set_bounds(0,1)
    ax2.set_yticks([0,.2,.4,.6,.8,1])
    ax2.set_yticklabels([0,20,40,60,80,100])
    ax2.set_ylabel('% Routine')
    
    
    
    ##########################################
    # 2: speed
    ax3= fig.add_subplot(gs[2])
    
    plot_group_event_effect(root, ax3, ProfilesCtrl1, nPre=nPre1, nPost=nPost1,TaskParamToPlot=TaskParamToPlot3)    
    ax3.set_ylim([-1,101])
    ax3.spines['left'].set_bounds(0,100)
    ax3.set_ylabel('Speed (cm/s)')

    
    
#     fig.align_ylabels([axes1[0],axes2[0],axes3[0],axes4[0]])
    #############################################
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#     AXES=(axes4[0],ax1,ax2,ax5,ax7,ax8,ax6)
#     OFFX=np.array([.07]*len(AXES))
#     OFFY=np.array([.01]*len(AXES))
#     OFFX[5]=0.03
#     OFFX[[0,1,2,4,6]]=0.05
    
#     add_panel_caption(axes=AXES, offsetX=OFFX, offsetY=OFFY)
    
    fig.savefig(os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','BreakEffect.pdf'),
                format='pdf', bbox_inches='tight')
    
    plt.show()
    plt.close('all')
    matplotlib.rcdefaults()